# Lecture 7 - Valuation in Practices

In this notebook, we build a complete valuation workflow for Tesla and connect it to ESG policy risk.

What you will practice:

- translating business assumptions into forecast tables,
- linking revenue, gross margin, operating expense, and reinvestment,
- building FCFF-based DCF valuation and sensitivity analysis,
- tracing how tradable regulatory credits affect ratios and intrinsic value.


## Concept Review

### 1. Enterprise Value vs Equity Value

- **Enterprise Value (EV)** is the value of the whole operating business.
- **Equity Value** is the value attributable to shareholders.
- In DCF with FCFF, we usually estimate **EV first**, then adjust for debt/cash/other claims if needed.

### 2. FCFF-based DCF

- Forecast operating performance and reinvestment.
- Compute **FCFF** for each forecast year.
- Discount FCFF and terminal value back to present value.
- Valuation is highly sensitive to **discount rate** and **terminal growth rate**.

### 3. WACC / CAPM / Terminal Growth

- CAPM is used to explain the **cost of equity** conceptually.
- Terminal value often dominates total value, so small changes in `WACC` or `g` can create large valuation changes.

### 4. ESG / Carbon-Policy Link to Valuation

- Tradable regulatory credits are a policy-driven revenue stream.
- This is a concrete bridge from **ESG / emissions regulation** to **accounting numbers** and **intrinsic value**.

## Initial Data Preparation

In this stage, we prepare the data used by the valuation model.

Practical workflow for this lesson:

- Load the prepared local datasets (financial statements, market data, and modeling inputs).
- Separate **core inputs** (historical anchors + assumptions) from **formula-generated tables**.
- Print the prepared inputs before building the model.

This keeps the modeling process stable and easy to follow in class.


In [1]:
# Setup: local data, helper functions, and model utilities
from pathlib import Path
import json
import sys
import importlib
import numpy as np
import pandas as pd
from IPython.display import display

DATA_DIR = Path('materials/data')
L7_DATA_DIR = DATA_DIR / 'lecture7_valuation_practices'

income = pd.read_excel(DATA_DIR / 'tesla_income_stmt.xlsx', index_col=0)
balance = pd.read_excel(DATA_DIR / 'tesla_balance_sheet.xlsx', index_col=0)
tsla_px = pd.read_excel(DATA_DIR / 'tesla_daily_price.xlsx')
sp500 = pd.read_csv(DATA_DIR / 'sp500_index_fred.csv')
us10y = pd.read_csv(DATA_DIR / 'us_10y_treasury_yield_dgs10.csv')

for df in [income, balance]:
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)
for df in [tsla_px, sp500, us10y]:
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', inplace=True)

base_year = income.index.max()
prev_year = income.index[-2]


def dcf_pv(cashflows, discount_rate):
    cashflows = np.asarray(cashflows, dtype=float)
    t = np.arange(1, len(cashflows) + 1)
    return float((cashflows / (1 + discount_rate) ** t).sum())


def terminal_value_growing_perpetuity(fcf_t, discount_rate, g):
    return float(fcf_t * (1 + g) / (discount_rate - g))


def show_df(df, digits=2):
    display(df.round(digits))


def l7_read_csv(filename, **kwargs):
    return pd.read_csv(L7_DATA_DIR / filename, **kwargs)


L7_CONSTANTS = json.loads((L7_DATA_DIR / 'lecture7_constants.json').read_text(encoding='utf-8'))
L7_CORE_INPUTS = json.loads((L7_DATA_DIR / 'lecture7_core_model_inputs.json').read_text(encoding='utf-8'))

sys.path.append(str(Path('materials').resolve()))
l7m = importlib.import_module('lecture7_core_model')


def l7_build_model(core):
    return l7m.build_workbook_style_model(core, dcf_pv_func=dcf_pv, terminal_value_func=terminal_value_growing_perpetuity)


print('Base year:', base_year.date())
print('Data folder:', DATA_DIR)
print('Model input file:', L7_DATA_DIR / 'lecture7_core_model_inputs.json')


Base year: 2025-12-31
Data folder: materials\data
Model input file: materials\data\lecture7_valuation_practices\lecture7_core_model_inputs.json


### Prepared Inputs Snapshot

This section prints the key inputs used to build our Python valuation model:

- historical anchor values,
- forecast targets and growth paths,
- margin and reinvestment assumptions,
- valuation assumptions.

All downstream tables (revenue, margins, opex, FCFF, and DCF) are generated from these inputs in Python.


In [2]:
# Build the valuation model from a compact set of prepared inputs
core = L7_CORE_INPUTS

revenue_inputs = pd.DataFrame({
    'Base 2025 (bn)': pd.Series(core['revenue']['base_segment_revenue_billion']),
    'Target (period 10, bn)': pd.Series(core['revenue']['target_revenue_billion_period_10']),
})
revenue_inputs.loc['robotaxi_launch_revenue'] = [np.nan, core['revenue']['robotaxi']['launch_revenue_billion']]

path_inputs = pd.DataFrame({
    'Automotive growth': core['revenue']['automotive_growth_path_period_1_to_10'],
    'Sales-to-capital': core['reinvestment']['sales_to_capital_path_period_1_to_10'],
}, index=[f'Period {i}' for i in range(1, 11)])

margin_inputs = pd.DataFrame({
    'Base': {
        'Automotive': core['gross_margin']['automotive_margin_base'],
        'Energy': core['gross_margin']['energy_margin_base'],
        'Software': core['gross_margin']['software_margin_base'],
        'Robotaxi': core['gross_margin']['robotaxi_margin_launch'],
    },
    'Target': {
        'Automotive': core['gross_margin']['automotive_margin_target'],
        'Energy': core['gross_margin']['energy_margin_target'],
        'Software': core['gross_margin']['software_margin_target'],
        'Robotaxi': core['gross_margin']['robotaxi_margin_target'],
    }
})

valuation_inputs = pd.Series({
    'Tax rate': core['valuation']['tax_rate'],
    'Risk-free rate': core['valuation']['risk_free_rate'],
    'Equity risk premium': core['valuation']['equity_risk_premium'],
    'Beta': core['valuation']['beta'],
    'Cost of debt': core['valuation']['cost_of_debt'],
    'Forecast discount rate': core['valuation']['forecast_discount_rate_workbook'],
    'Terminal discount rate': core['valuation']['terminal_discount_rate'],
    'Terminal growth rate': core['valuation']['terminal_growth_rate'],
    'Tradable credit revenue (2025, bn)': core['policy_credit']['tradable_credit_revenue_2025_billion'],
}, name='Value')

print('Prepared inputs used by the Python model')
show_df(revenue_inputs, 4)
show_df(path_inputs, 4)
show_df(margin_inputs, 4)
show_df(valuation_inputs.to_frame(), 4)

l7_model = l7_build_model(core)


Prepared inputs used by the Python model


,Base 2025 (bn),"Target (period 10, bn)"
automotive,69.53,300.0
energy,12.77,80.0
software,12.53,40.0
robotaxi,0.00,80.0
robotaxi_launch_revenue,NaN,11.4


,Automotive growth,Sales-to-capital
Period 1,0.25,5.0
Period 2,0.24,5.0
Period 3,0.23,4.0
Period 4,0.22,4.0
Period 5,0.20,3.0
Period 6,0.16,2.0
Period 7,0.12,2.0
Period 8,0.08,2.0
Period 9,0.08,2.0
Period 10,0.08,2.0


,Base,Target
Automotive,0.178,0.20
Energy,0.298,0.38
Software,0.075,0.22
Robotaxi,0.050,0.25


,Value
Tax rate,0.2400
Risk-free rate,0.0408
Equity risk premium,0.1200
Beta,2.0500
Cost of debt,0.0500
Forecast discount rate,0.0929
Terminal discount rate,0.0835
Terminal growth rate,0.0422
"Tradable credit revenue (2025, bn)",1.9330


### Historical Financial Performance

This code summarizes the key labeled historical changes shown on the slide:

- revenue mix shift (Automotive vs Energy / Services)
- operating expense increase (especially R&D)

These observations motivate later forecasting assumptions.

In [3]:
# Historical labels used in the teaching slide (USD billions), loaded from local teaching tables
segment_rev = l7_read_csv('page6_segment_revenue_labels.csv').set_index('Segment')
segment_rev['Change (B)'] = segment_rev['2025'] - segment_rev['2023']
segment_rev['Growth (%)'] = (segment_rev['2025'] / segment_rev['2023'] - 1) * 100
show_df(segment_rev, 2)

opex_labeled = l7_read_csv('page6_opex_labels.csv').set_index('Item')
opex_labeled['Change (B)'] = opex_labeled['2025'] - opex_labeled['2023']
opex_labeled['Growth (%)'] = (opex_labeled['2025'] / opex_labeled['2023'] - 1) * 100
show_df(opex_labeled, 2)


,2023,2025,Change (B),Growth (%)
Segment,,,,
Automotive,82.42,69.53,-12.89,-15.64
Energy and Storage,6.04,12.77,6.73,111.42
Services and Other,8.32,12.53,4.21,50.60


,2023,2025,Change (B),Growth (%)
Item,,,,
R&D,3.97,6.41,2.44,61.46
SG&A,4.80,5.83,1.03,21.46
Other Opex,NaN,0.50,NaN,NaN


## Model Building

### Revenue Forecast Assumptions and Revenue Path

Teaching objective:

- translate qualitative business assumptions into a structured forecast table,
- separate **assumptions** from **mechanical calculations**, and
- understand how target-anchored rows affect implied growth rates.


In [4]:
# Revenue forecast generated from the prepared inputs
rev_tbl_full = l7_model['revenue'].copy()
base_row = l7_model['base_row'].copy()

base_rev = {
    'Total Revenue': float(base_row['Total Revenue']),
    'Automotive': float(base_row['Automotive']),
    'Energy': float(base_row['Energy']),
    'Software': float(base_row['Software']),
    'Robotaxi': float(base_row['Robotaxi']),
}
base_total_rev = base_rev['Total Revenue']

rev_tbl = rev_tbl_full.rename(columns={'Total Revenue': 'Total Revenue (Billion)', 'growth rate': 'Automotive growth'}).copy()
rev_tbl[['Total Revenue (Billion)', 'Automotive', 'Energy', 'Software', 'Robotaxi']] = (
    rev_tbl[['Total Revenue (Billion)', 'Automotive', 'Energy', 'Software', 'Robotaxi']].round(2)
)
rev_tbl['Automotive growth'] = rev_tbl['Automotive growth'].map(lambda x: f'{x:.2%}')

base_row_display = pd.DataFrame([{
    'Forecast Period': base_row['Forecast Period'],
    'Total Revenue (Billion)': base_rev['Total Revenue'],
    'Automotive': base_rev['Automotive'],
    'Automotive growth': '',
    'Energy': base_rev['Energy'],
    'Software': base_rev['Software'],
    'Robotaxi': base_rev['Robotaxi'],
}])
show_df(pd.concat([base_row_display, rev_tbl], ignore_index=True), 2)

cagrs = pd.Series({
    'Automotive (10y)': (rev_tbl_full.iloc[-1]['Automotive'] / base_rev['Automotive']) ** (1/10) - 1,
    'Energy (10y)': (rev_tbl_full.iloc[-1]['Energy'] / base_rev['Energy']) ** (1/10) - 1,
    'Software (10y)': (rev_tbl_full.iloc[-1]['Software'] / base_rev['Software']) ** (1/10) - 1,
    'Robotaxi (6y, p4->p10)': (rev_tbl_full.iloc[-1]['Robotaxi'] / rev_tbl_full.loc[3, 'Robotaxi']) ** (1/6) - 1,
}, name='Implied CAGR')
show_df(cagrs.to_frame(), 4)


,Forecast Period,Total Revenue (Billion),Automotive,Automotive growth,Energy,Software,Robotaxi
0,Base Year (2025),94.83,69.53,,12.77,12.53,0.00
1,1,116.33,86.91,25.00%,15.34,14.07,0.00
2,2,142.01,107.77,24.00%,18.43,15.80,0.00
3,3,172.45,132.56,23.00%,22.14,17.75,0.00
4,4,219.66,161.72,22.00%,26.60,19.93,11.40
5,5,264.19,194.07,20.00%,31.96,22.39,15.77
6,6,310.49,225.12,16.00%,38.40,25.14,21.83
7,7,356.70,252.13,12.00%,46.13,28.24,30.20
8,8,401.23,272.30,8.00%,55.43,31.71,41.79
9,9,454.11,294.09,8.00%,66.59,35.62,57.82


,Implied CAGR
Automotive (10y),0.1574
Energy (10y),0.2014
Software (10y),0.1231
"Robotaxi (6y, p4->p10)",0.3837


### Gross Margin Modeling

This section builds the gross margin model directly in Python.

Teaching focus:

- compute segment-level gross profit from forecast revenues,
- apply margin paths by segment (Automotive / Energy / Software / Robotaxi),
- aggregate to total gross profit and total gross margin.

Implementation note:

- The provided teaching data uses a linked-revenue assumption for the software gross-profit line.  
- We keep that assumption in code so the calculations remain consistent across sections.


In [5]:
# Gross margin model built from the prepared inputs
gross_margin_sheet_py = l7_model['gross_margin'].copy()

gm_margin_view = gross_margin_sheet_py[[
    'Forecast Period', 'Auto GM', 'Energy GM', 'Software GM', 'Robotaxi GM', 'Total gross margin'
]].rename(columns={'Robotaxi GM': 'Robotaxi GM (from launch onward)'})
show_df(gm_margin_view, 4)

gm_profit_view = gross_margin_sheet_py[[
    'Forecast Period', 'Auto GP', 'Energy GP', 'Software GP (workbook logic)', 'Robotaxi GP', 'Total gross profit'
]].rename(columns={'Software GP (workbook logic)': 'Software GP'})
show_df(gm_profit_view, 4)


,Forecast Period,Auto GM,Energy GM,Software GM,Robotaxi GM (from launch onward),Total gross margin
0,1,0.1780,0.3062,0.0895,NaN,0.1852
1,2,0.1835,0.3153,0.1040,NaN,0.1937
2,3,0.1890,0.3244,0.1185,NaN,0.2022
3,4,0.1931,0.3335,0.1330,0.0500,0.2013
4,5,0.2000,0.3426,0.1475,0.0833,0.2112
5,6,0.2000,0.3518,0.1620,0.1167,0.2168
6,7,0.2000,0.3609,0.1765,0.1500,0.2236
7,8,0.2000,0.3700,0.1910,0.1833,0.2323
8,9,0.2000,0.3791,0.2055,0.2167,0.2428
9,10,0.2000,0.3800,0.2200,0.2500,0.2560


,Forecast Period,Auto GP,Energy GP,Software GP,Robotaxi GP,Total gross profit
0,1,15.4704,4.6977,1.3734,0.0000,21.5415
1,2,19.7761,5.8118,1.9172,0.0000,27.5051
2,3,25.0536,7.1841,2.6244,0.0000,34.8621
3,4,31.2325,8.8734,3.5387,0.5700,44.2146
4,5,38.8133,10.9518,4.7148,1.3145,55.7943
5,6,45.0234,13.5074,6.2211,2.5463,67.2982
6,7,50.4262,16.6482,8.1429,4.5299,79.7472
7,8,54.4603,20.5062,10.5865,7.6607,93.2137
8,9,58.8171,25.2430,13.6841,12.5271,110.2713
9,10,60.0000,30.4000,17.6000,20.0000,128.0000


### Operating Expense Modeling

This section builds the operating expense model directly in Python.

Teaching focus:

- convert gross profit into EBIT by modeling operating expenses,
- understand how R&D, SG&A, and other expenses evolve over time,
- connect operating assumptions to FCFF inputs.


In [6]:
# Operating expense model built from the prepared inputs
operating_expense_sheet_py = l7_model['operating_expense'].copy()

opex_view = operating_expense_sheet_py[[
    'Forecast Period', 'R&D', 'R&D growth', 'SG&A', 'Restructuring and other',
    'Total operating expense', 'Operating margin', 'Operating profit (EBIT)'
]].copy()
show_df(opex_view, 4)


,Forecast Period,R&D,R&D growth,SG&A,Restructuring and other,Total operating expense,Operating margin,Operating profit (EBIT)
0,1,8.7296,0.3617,5.834,0.494,15.0576,0.0557,6.4839
1,2,11.5134,0.3189,5.834,0.494,17.8414,0.0681,9.6636
2,3,14.7745,0.2832,5.834,0.494,21.1025,0.0798,13.7596
3,4,18.2618,0.2360,5.834,0.494,24.5898,0.0893,19.6248
4,5,21.8538,0.1967,5.834,0.494,28.1818,0.1045,27.6125
5,6,25.4359,0.1639,5.834,0.494,31.7639,0.1144,35.5343
6,7,28.9103,0.1366,5.834,0.494,35.2383,0.1248,44.5089
7,8,32.3795,0.1200,5.834,0.494,38.7075,0.1358,54.5062
8,9,35.7794,0.1050,5.834,0.494,42.1074,0.1501,68.1640
9,10,39.5362,0.1050,5.834,0.494,45.8642,0.1643,82.1358


### Reinvestment, FCFF, and Capital Efficiency

Teaching objective:

- connect operating margin assumptions to EBIT,
- connect revenue growth to reinvestment,
- compute FCFF and inspect implied capital-efficiency assumptions.

In [7]:
# Reinvestment and FCFF generated from the model inputs
reinvestment_tbl = l7_model['reinvestment'].copy()
fcff_tbl = l7_model['fcff'].copy()

show_df(reinvestment_tbl, 4)
show_df(fcff_tbl[['Forecast Period', 'Total Revenues', 'Operating Margin', 'EBIT(1-t)', 'Reinvestment', 'FCFF']], 4)


,Forecast Period,Sales to Capital,Revenue growth,Reinvestment
0,1,5.0,21.4967,4.2993
1,2,5.0,25.6810,5.1362
2,3,4.0,30.4449,7.6112
3,4,4.0,47.2076,11.8019
4,5,3.0,44.5299,14.8433
5,6,2.0,46.2954,23.1477
6,7,2.0,46.2163,23.1082
7,8,2.0,44.5240,22.2620
8,9,2.0,52.8822,26.4411
9,10,2.0,45.8921,22.9460


,Forecast Period,Total Revenues,Operating Margin,EBIT(1-t),Reinvestment,FCFF
0,1,116.3267,0.0557,4.9277,4.2993,0.6284
1,2,142.0076,0.0681,7.3444,5.1362,2.2082
2,3,172.4526,0.0798,10.4573,7.6112,2.8461
3,4,219.6601,0.0893,14.9149,11.8019,3.1130
4,5,264.1900,0.1045,20.9855,14.8433,6.1422
5,6,310.4854,0.1144,27.0061,23.1477,3.8584
6,7,356.7017,0.1248,33.8268,23.1082,10.7186
7,8,401.2258,0.1358,41.4247,22.2620,19.1627
8,9,454.1079,0.1501,51.8046,26.4411,25.3635
9,10,500.0000,0.1643,62.4232,22.9460,39.4772


## Results Display

### WACC, Terminal Growth, DCF, and Sensitivity

Teaching objective:

- estimate a **practice beta** from market data,
- compare slide assumptions vs code-derived values,
- understand why terminal value and discount rate dominate valuation outcomes.

In [8]:
# Valuation ingredients, intrinsic value summary, and sensitivity matrix
px = tsla_px[['Date', 'Close']].rename(columns={'Close': 'TSLA_Close'})
sp = sp500[['Date', 'SP500']]
rf = us10y[['Date', 'DGS10']]
mkt = px.merge(sp, on='Date', how='inner').merge(rf, on='Date', how='inner').sort_values('Date')
mkt['DGS10'] = mkt['DGS10'].ffill()
mkt['r_tsla'] = mkt['TSLA_Close'].pct_change()
mkt['r_mkt'] = mkt['SP500'].pct_change()
mkt['rf_daily'] = (mkt['DGS10'] / 100) / 252
mkt['excess_tsla'] = mkt['r_tsla'] - mkt['rf_daily']
mkt['excess_mkt'] = mkt['r_mkt'] - mkt['rf_daily']
reg = mkt.dropna(subset=['excess_tsla', 'excess_mkt'])
beta_est = reg['excess_tsla'].cov(reg['excess_mkt']) / reg['excess_mkt'].var()
rf_latest = float(mkt['DGS10'].dropna().iloc[-1] / 100)

ni_2025 = float(income.loc[base_year, 'Net Income Including Noncontrolling Interests'])
eq_2025 = float(balance.loc[base_year, 'Common Stock Equity'])
eq_2024 = float(balance.loc[prev_year, 'Common Stock Equity'])
roe_2025 = ni_2025 / ((eq_2025 + eq_2024) / 2)

valuation_from_core = l7_model['valuation']
iv_from_core = valuation_from_core['intrinsic_value_components']

valuation_summary = pd.Series({
    'Estimated beta (from local daily data)': beta_est,
    'Latest local 10Y Treasury': rf_latest,
    'ROE FY2025': roe_2025,
    'Tax rate (model input)': float(L7_CORE_INPUTS['valuation']['tax_rate']),
    'Forecast discount rate': float(L7_CORE_INPUTS['valuation']['forecast_discount_rate_workbook']),
    'Terminal discount rate': float(L7_CORE_INPUTS['valuation']['terminal_discount_rate']),
    'Terminal growth rate': float(L7_CORE_INPUTS['valuation']['terminal_growth_rate']),
}, name='Value')
show_df(valuation_summary.to_frame(), 4)
show_df(iv_from_core.to_frame('Intrinsic Value (USD bn)'), 4)

fcff = fcff_tbl['FCFF'].to_numpy(dtype=float)
center_r = float(L7_CORE_INPUTS['valuation']['terminal_discount_rate'])
center_g = float(L7_CORE_INPUTS['valuation']['terminal_growth_rate'])
r_grid = [center_r - 0.01, center_r - 0.005, center_r, center_r + 0.005, center_r + 0.01]
g_grid = [center_g - 0.005, center_g, center_g + 0.005]

iv_matrix = pd.DataFrame(index=[f'{r:.2%}' for r in r_grid], columns=[f'{g:.2%}' for g in g_grid], dtype=float)
for r_ in r_grid:
    for g_ in g_grid:
        pv_f = dcf_pv(fcff, r_)
        tv_ = terminal_value_growing_perpetuity(fcff[-1], r_, g_)
        pv_tv_ = tv_ / (1 + r_) ** len(fcff)
        iv_matrix.loc[f'{r_:.2%}', f'{g_:.2%}'] = pv_f + pv_tv_
show_df(iv_matrix, 2)


,Value
Estimated beta (from local daily data),2.0525
Latest local 10Y Treasury,0.0418
ROE FY2025,0.0497
Tax rate (model input),0.2400
Forecast discount rate,0.0929
Terminal discount rate,0.0835
Terminal growth rate,0.0422


,Intrinsic Value (USD bn)
Terminal Value,996.2014
PV(Terminal Value),446.7432
PV(CF over Forecast Period),55.7831
Value of Operating Assets,502.5263


,3.72%,4.22%,4.72%
7.35%,619.17,710.93,837.58
7.85%,527.53,594.23,682.23
8.35%,456.27,506.43,570.40
8.85%,399.41,438.15,486.27
9.35%,353.08,383.65,420.83


### ESG / Policy Channel: ZEV Tradable Credits

This short step extracts the visible Tesla transfer rows from the slide screenshot to make the regulatory-credit mechanism more concrete.

In [9]:
# Tesla's visible ZEV credit transfers (from the teaching slide table)
tesla_visible_transfers = l7_read_csv('zev_visible_transfers_model_year_2023.csv')
show_df(tesla_visible_transfers, 0)
print('Visible Tesla-transfer credits total:', int(tesla_visible_transfers['Credits'].sum()))


,Transferee,Credits
0,Fiat Chrysler (Stellantis),69319
1,Ford,35000
2,Honda,7777
3,Mazda,30345
4,Mitsubishi,1288
5,Subaru,3376
6,Toyota,123414


Visible Tesla-transfer credits total: 270519


### Tradable Credits -> Ratios -> Intrinsic Value

Teaching objective:

- quantify the materiality of tradable-credit revenue,
- propagate the shock to profitability and liquidity ratios,
- compare intrinsic value **with vs without** tradable-credit revenue.

This section is the main bridge between **carbon/ESG policy context** and **valuation practice**.

In [10]:
# Tradable-credit scenario: ratios and intrinsic value impact
credit_rev = float(L7_CONSTANTS['credit_revenue_usd'])
rev = float(income.loc[base_year, 'Total Revenue'])
ni_incl_nci = float(income.loc[base_year, 'Net Income Including Noncontrolling Interests'])
ni_common = float(income.loc[base_year, 'Net Income Common Stockholders'])

materiality = pd.Series({
    'Credit revenue (USD m)': credit_rev / 1e6,
    'Share of total revenue (%)': credit_rev / rev * 100,
    'Share of net income incl. NCI (%)': credit_rev / ni_incl_nci * 100,
    'Share of net income common (%)': credit_rev / ni_common * 100,
}, name='Value')
show_df(materiality.to_frame(), 2)

cash_ce = float(balance.loc[base_year, 'Cash And Cash Equivalents'])
retained = float(balance.loc[base_year, 'Retained Earnings'])
ebit = float(income.loc[base_year, 'EBIT'])
scenario_tbl = pd.DataFrame([
    {
        'Credit retention': f'{r:.0%}',
        'Revenue (bn)': (rev - credit_rev * (1 - r)) / 1e9,
        'EBIT (bn)': (ebit - credit_rev * (1 - r)) / 1e9,
        'Net income common (bn)': (ni_common - credit_rev * (1 - r)) / 1e9,
        'Cash (bn)': (cash_ce - credit_rev * (1 - r)) / 1e9,
        'Retained earnings (bn)': (retained - credit_rev * (1 - r)) / 1e9,
    }
    for r in [1.0, 0.5, 0.0]
])
show_df(scenario_tbl, 2)

cogs = float(income.loc[base_year, 'Cost Of Revenue'])
gp = float(income.loc[base_year, 'Gross Profit'])
ca = float(balance.loc[base_year, 'Current Assets'])
cl = float(balance.loc[base_year, 'Current Liabilities'])
inv = float(balance.loc[base_year, 'Inventory'])
ar = float(balance.loc[base_year, 'Accounts Receivable'])
ap = float(balance.loc[base_year, 'Accounts Payable'])
inv_avg = float((balance.loc[base_year, 'Inventory'] + balance.loc[prev_year, 'Inventory']) / 2)
ap_avg = float((balance.loc[base_year, 'Accounts Payable'] + balance.loc[prev_year, 'Accounts Payable']) / 2)
ar_avg = float((balance.loc[base_year, 'Accounts Receivable'] + balance.loc[prev_year, 'Accounts Receivable']) / 2)
rev_after = rev - credit_rev
ratio_tbl = pd.DataFrame([
    ('Gross Margin', gp / rev, (gp - credit_rev) / rev_after),
    ('Operating Profit Margin', ebit / rev, (ebit - credit_rev) / rev_after),
    ('Net Profit Margin', ni_common / rev, (ni_common - credit_rev) / rev_after),
    ('Current Ratio', ca / cl, (ca - credit_rev) / cl),
    ('Quick Ratio', (ca - inv) / cl, (ca - credit_rev - inv) / cl),
    ('Cash Ratio', cash_ce / cl, (cash_ce - credit_rev) / cl),
    ('Inventory Turnover', cogs / inv_avg, cogs / inv_avg),
    ('A/P Turnover', cogs / ap_avg, cogs / ap_avg),
    ('A/R Turnover', rev / ar_avg, rev_after / ar_avg),
], columns=['Metric', 'Before', 'After']).set_index('Metric')
show_df(ratio_tbl, 2)

base_auto_with = base_rev['Automotive']
base_auto_without = base_auto_with - float(L7_CONSTANTS['credit_revenue_for_forecast_billion'])
auto_cagr_compare = pd.Series({
    'Automotive CAGR with credit': (300 / base_auto_with) ** (1 / 10) - 1,
    'Automotive CAGR without credit': (300 / base_auto_without) ** (1 / 10) - 1,
}, name='Implied CAGR')
show_df(auto_cagr_compare.to_frame(), 4)

iv_compare = l7_read_csv('valuation_without_credit_intrinsic_value_comparison.csv').set_index('Metric')
iv_compare['Change'] = iv_compare['Without credit'] - iv_compare['With credit']
iv_compare['Change (%)'] = iv_compare['Change'] / iv_compare['With credit'] * 100
show_df(iv_compare, 2)


,Value
Credit revenue (USD m),1993.00
Share of total revenue (%),2.10
Share of net income incl. NCI (%),51.70
Share of net income common (%),52.53


,Credit retention,Revenue (bn),EBIT (bn),Net income common (bn),Cash (bn),Retained earnings (bn)
0,100%,94.83,5.62,3.79,16.51,39.00
1,50%,93.83,4.62,2.80,15.52,38.01
2,0%,92.83,3.62,1.80,14.52,37.01


,Before,After
Metric,,
Gross Margin,0.18,0.16
Operating Profit Margin,0.06,0.04
Net Profit Margin,0.04,0.02
Current Ratio,2.16,2.10
Quick Ratio,1.77,1.71
Cash Ratio,0.52,0.46
Inventory Turnover,6.37,6.37
A/P Turnover,6.02,6.02
A/R Turnover,21.09,20.64


,Implied CAGR
Automotive CAGR with credit,0.1574
Automotive CAGR without credit,0.1607


,With credit,Without credit,Change,Change (%)
Metric,,,,
Terminal Value,996.20,825.86,-170.34,-17.10
PV(Terminal Value),446.74,370.35,-76.39,-17.10
PV(CF over forecast period),55.78,36.03,-19.75,-35.41
Value of operating assets,502.53,406.38,-96.15,-19.13


### Lesson Summary

In this lesson, you built a valuation workflow directly in Python.

What you practiced:

1. Organizing prepared inputs (historical anchors + assumptions)
2. Building revenue, margin, and operating expense forecasts
3. Translating forecasts into reinvestment and FCFF
4. Estimating intrinsic value with DCF and sensitivity analysis
5. Tracing ESG / policy impacts (tradable credits) into ratios and valuation

A strong modeling habit is to keep the workflow in this order:

- prepared inputs,
- model construction,
- result display,
- scenario analysis.

This makes the model easier to read, easier to debug, and easier to update when assumptions change.
